In [24]:
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score

In [25]:
df_train_labels_original = pd.read_csv('train_labels.csv',low_memory=False,index_col='building_id')
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, index_col='building_id', dtype= {
'geo_level_1_id':'uint8', 'geo_level_2_id':'uint16', 'geo_level_3_id':'uint16', 'count_floors_pre_eq':'uint8','age':'uint16', 'area_percentage':'uint16', 'height_percentage':'uint16', 
'land_surface_condition':'category', 'foundation_type':'category', 'roof_type':'category', 'ground_floor_type':'category', 'other_floor_type':'category', 'position':'category','plan_configuration':'category', 
'has_superstructure_adobe_mud':'bool', 'has_superstructure_mud_mortar_stone':'bool','has_superstructure_stone_flag':'bool', 'has_superstructure_cement_mortar_stone':'bool', 'has_superstructure_mud_mortar_brick':'bool', 'has_superstructure_cement_mortar_brick':'bool', 'has_superstructure_timber':'bool', 'has_superstructure_bamboo':'bool', 'has_superstructure_rc_non_engineered':'bool', 'has_superstructure_rc_engineered':'bool', 'has_superstructure_other':'bool', 
'legal_ownership_status':'category', 'count_families':'uint16', 
'has_secondary_use':'bool', 'has_secondary_use_agriculture':'bool', 'has_secondary_use_hotel':'bool', 'has_secondary_use_rental':'bool', 'has_secondary_use_institution':'bool', 'has_secondary_use_school':'bool', 'has_secondary_use_industry':'bool', 'has_secondary_use_health_post':'bool', 'has_secondary_use_gov_office':'bool', 'has_secondary_use_use_police':'bool', 'has_secondary_use_other':'bool',})
df_test_values_original = pd.read_csv('test_values.csv',low_memory=False, index_col='building_id', dtype= {
'geo_level_1_id':'uint8', 'geo_level_2_id':'uint16', 'geo_level_3_id':'uint16', 'count_floors_pre_eq':'uint8','age':'uint16', 'area_percentage':'uint16', 'height_percentage':'uint16', 
'land_surface_condition':'category', 'foundation_type':'category', 'roof_type':'category', 'ground_floor_type':'category', 'other_floor_type':'category', 'position':'category','plan_configuration':'category', 
'has_superstructure_adobe_mud':'bool', 'has_superstructure_mud_mortar_stone':'bool','has_superstructure_stone_flag':'bool', 'has_superstructure_cement_mortar_stone':'bool', 'has_superstructure_mud_mortar_brick':'bool', 'has_superstructure_cement_mortar_brick':'bool', 'has_superstructure_timber':'bool', 'has_superstructure_bamboo':'bool', 'has_superstructure_rc_non_engineered':'bool', 'has_superstructure_rc_engineered':'bool', 'has_superstructure_other':'bool', 
'legal_ownership_status':'category', 'count_families':'uint16', 
'has_secondary_use':'bool', 'has_secondary_use_agriculture':'bool', 'has_secondary_use_hotel':'bool', 'has_secondary_use_rental':'bool', 'has_secondary_use_institution':'bool', 'has_secondary_use_school':'bool', 'has_secondary_use_industry':'bool', 'has_secondary_use_health_post':'bool', 'has_secondary_use_gov_office':'bool', 'has_secondary_use_use_police':'bool', 'has_secondary_use_other':'bool',})

In [26]:
train_values_subset = pd.get_dummies(df_train_values_original)
train_labels_subset = df_train_labels_original['damage_grade']

validation_size = df_train_values_original.index.size - df_test_values_original.index.size

In [27]:
train_values, validation_values = (train_values_subset.iloc[0:173733], train_values_subset.iloc[173733:-1])
train_labels, validation_labels = (train_labels_subset.iloc[0:173733], train_labels_subset.iloc[173733:-1])

In [28]:
redNeuronalPredictionsTrain = pd.read_csv('ensamble_predictions_feda.csv', index_col=0)
redNeuronalPredictionsTest = pd.read_csv('ensamble_submition_feda.csv', index_col=0)

In [29]:
xgBoostPredictionsTest = pd.read_csv('xgBoostPredictionProba.csv', index_col=0)
xgBoostPredictionsTrain = pd.read_csv('xgBoostPredictionProbaTrain.csv', index_col=0)

In [30]:
rndForestPredictionsTest = pd.read_csv('randomForestPredictionProba', index_col=0)
rndForestPredictionsTrain = pd.read_csv('randomForestPredictionProbaTrain.csv', index_col=0)

In [31]:
logRegPredictionsTest = pd.read_csv('LogisticRegresionProbaTest.csv', index_col=0)
logRegPredictionsTrain = pd.read_csv('LogisticRegresionProbaTrain.csv', index_col=0)

In [40]:
trainData = pd.concat([rndForestPredictionsTrain , xgBoostPredictionsTrain, redNeuronalPredictionsTrain, logRegPredictionsTrain], axis=1 )

In [33]:
testData = pd.concat([rndForestPredictionsTest , xgBoostPredictionsTest, redNeuronalPredictionsTest, logRegPredictionsTest], axis=1 )

In [34]:
# esta a fuerza bruta, pero lleva poco tiempo en definitiva
scores = []
for a in [0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13]:
    for b in [0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33]:
        for c in [0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53]:
            for d in [0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13]:
                if (a+b+c+d != 1): continue
                else:
                    trainData = pd.concat([rndForestPredictionsTrain , xgBoostPredictionsTrain, redNeuronalPredictionsTrain, logRegPredictionsTrain], axis=1 )
                    trainData = trainData.assign(dg1 = lambda x: (x.rndf1 * a) + (x.neural1 * b) + (x.xgb1 * c) + (x.logReg1 * d))
                    trainData = trainData.assign(dg2 = lambda x: (x.rndf2 * a) + (x.neural2 * b) + (x.xgb2 * c) + (x.logReg2 * d))
                    trainData = trainData.assign(dg3 = lambda x: (x.rndf3 * a) + (x.neural3 * b) + (x.xgb3 * c) + (x.logReg3 * d))
                    trainData = trainData[['dg1', 'dg2', 'dg3']]
                    dgTest = []
                    for x in range(0, 86867):
                        dgTest.append(1 if ((trainData.dg1[x] > trainData.dg2[x]) and (trainData.dg1[x] > trainData.dg2[x])) else 2 if ((trainData.dg2[x] > trainData.dg3[x]) and (trainData.dg2[x] > trainData.dg1[x])) else 3)
                    score = f1_score(validation_labels, dgTest, average='micro')
                    scores.append([score, a, b, c, d])

In [35]:
max(scores, key= lambda x: x[0])

[0.7505151553524353, 0.07, 0.33, 0.53, 0.07]

[0.7488574487434814, 0.1, 0.3, 0.5, 0.1]
[0.7505151553524353, 0.07, 0.33, 0.53, 0.07]

In [36]:
testData = testData.assign(dg1 = lambda x: (x.rndf1 * 0.25) + (x.neural1 * 0.25) + (x.xgb1 * 0.4) + (x.logReg1 * 0.1))
testData = testData.assign(dg2 = lambda x: (x.rndf2 * 0.25) + (x.neural2 * 0.25) + (x.xgb2 * 0.4) + (x.logReg2 * 0.1))
testData = testData.assign(dg3 = lambda x: (x.rndf3 * 0.25) + (x.neural3 * 0.25) + (x.xgb3 * 0.4) + (x.logReg3 * 0.1))
testData = testData[['dg1', 'dg2', 'dg3']]

In [41]:
trainData = trainData.assign(dg1 = lambda x: (x.rndf1 * 0.07) + (x.neural1 * 0.33) + (x.xgb1 * 0.53) + (x.logReg1 * 0.07))
trainData = trainData.assign(dg2 = lambda x: (x.rndf2 * 0.07) + (x.neural2 * 0.33) + (x.xgb2 * 0.53) + (x.logReg2 * 0.07))
trainData = trainData.assign(dg3 = lambda x: (x.rndf3 * 0.07) + (x.neural3 * 0.33) + (x.xgb3 * 0.53) + (x.logReg3 * 0.07))
trainData = trainData[['dg1', 'dg2', 'dg3']]

In [42]:
dgTest = []
for x in range(0, 86867):
    dgTest.append(1 if ((trainData.dg1[x] > trainData.dg2[x]) and (trainData.dg1[x] > trainData.dg2[x])) else 2 if ((trainData.dg2[x] > trainData.dg3[x]) and (trainData.dg2[x] > trainData.dg1[x])) else 3)

In [43]:
f1_score(validation_labels, dgTest, average='micro')

0.7505151553524353

In [ ]:
dgFinal = []
for x in range(0, 86868):
    dgFinal.append(1 if ((testData.dg1[x] > testData.dg2[x]) and (testData.dg1[x] > testData.dg2[x])) else 2 if ((testData.dg2[x] > testData.dg3[x]) and (testData.dg2[x] > testData.dg1[x])) else 3)

In [ ]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=dgFinal,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.to_csv('submissionEnsamble.csv')
my_submission

0.74561110663428 primer intento ensamble con solver = 'lbfgs'